
# Challenge Problem 1 — 2.156 Artificial Intelligence and Machine Learning for Engineering Design (MIT, Fall 2025)

This notebook assembles and organizes the provided **Starter** and **Advanced** notebooks for linkage synthesis into a single workflow that:
- Sets up the environment and loads target curves
- Defines the mixed‑variable GA problem that searches over **structure + geometry**
- Seeds GA with random mechanisms
- Runs NSGA‑II
- Demonstrates gradient tools
- Visualizes and saves results

All code blocks below are taken from the course-provided notebooks and kept intact, only arranged for a coherent end‑to‑end run.


## 1) Setup
### 1.a Environment Setup (Advanced — Cell 0)

In [ ]:

import os
os.environ["JAX_PLATFORMS"] = "cpu"  # Disable GPU for JAX (Remove if you want to use GPU)

import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm.auto import tqdm, trange

# deteministic random numbers
np.random.seed(0)
random.seed(0)


### 1.b Load Target Curves (Advanced — Cell 2)

In [ ]:

target_curves = np.load('target_curves.npy')

# Plot all target curves

# Initialize a 2x3 subplot for plotting all target curves
fig, axs = plt.subplots(2, 3, figsize=(8, 5))

# Loop through the 6 target curves to plot them
for i in range(6):
    # Extract x and y coordinates from the target curve
    x_coords = np.array(target_curves[i])[:, 0]
    y_coords = np.array(target_curves[i])[:, 1]

    # Plot the curve on the respective subplot
    axs[i // 3, i % 3].plot(x_coords, y_coords, color='black', linewidth=3)

    # Set title for each subplot
    axs[i // 3, i % 3].set_title(f'Egg {i + 1}')

    # Ensure equal scaling for x and y axes
    axs[i // 3, i % 3].axis('equal')
    axs[i // 3, i % 3].axis('off')


### 1.c GA & Tools Imports (Advanced — Cell 4)

In [ ]:

from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.variable import Real, Integer, Choice, Binary
from pymoo.core.mixed import MixedVariableMating, MixedVariableGA, MixedVariableSampling, MixedVariableDuplicateElimination
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import FloatRandomSampling, Sampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.optimize import minimize

from LINKS.Optimization import DifferentiableTools, Tools


### 1.d Compile Optimization Tools (Advanced — Cell 5)

In [ ]:

PROBLEM_TOOLS = Tools( # we have to define this outside the class due to pymoo deepcopy limitations
            device='cpu' # device to run the optimization on
        )  
PROBLEM_TOOLS.compile() # compile the functions for faster runs


### 1.e Mixed‑Variable Problem Definition (Advanced — Cell 5)

In [ ]:

class mechanism_synthesis_optimization(ElementwiseProblem):

    # When intializing, set the mechanism size and target curve
    def __init__(self, target_curve, N = 5):
        self.N = N
        variables = dict()

        # The upper triangular portion of our NxN Connectivity Matrix consists of Nx(N-1)/2 boolean variables:
        for i in range(N):
            for j in range(i):
                variables["C" + str(j) + "_" + str(i)] = Binary()

        # We Delete C0_1 since we know node 1 is connected to the motor
        del variables["C0_1"]

        #Our position matrix consists of Nx2 real numbers (cartesian coordinate values) between 0 and 1
        for i in range(2*N):
            variables["X0" + str(i)] = Real(bounds=(0.0, 1.0))

        # Our node type vector consists of N boolean variables (fixed vs non-fixed)
        for i in range(N):
            variables["fixed_nodes" + str(i)] =  Binary(N)

        # Our target node is an integer between 1 and N-1, (any except the motor node).
        variables["target"] = Integer(bounds=(1,N-1))

        # Set up some variables in the problem class we inherit for pymoo
        # n_obj=number of objectives, n_constr=number of constraints
        # Our objectives are chamfer distance and material, and they both have constraints.
        super().__init__(vars=variables, n_obj=2, n_constr=2)

        # Store the target curve point cloud
        self.target_curve = target_curve


    def convert_1D_to_mech(self, x):
        N = self.N

        # Get target joints index
        target_idx = x["target"]

        # Build connectivity matrix from its flattened constitutive variables
        C = np.zeros((N,N))
        x["C0_1"] = 1

        for i in range(N):
            for j in range(i):
                # C[i,j] = x["C" + str(j) + "_" + str(i)]
                C[j,i] = x["C" + str(j) + "_" + str(i)]

        edges = np.array(np.where(C==1)).T
        
        # Reshape flattened position matrix to its proper Nx2 shape
        x0 = np.array([x["X0" + str(i)] for i in range(2*N)]).reshape([N,2])

        # Extract a list of Nodes that are fixed from boolean fixed_nodes vector
        fixed_joints = np.where(np.array([x["fixed_nodes" + str(i)] for i in range(N)]))[0].astype(int)

        #We fix the motor and original ground node as 0 and 1 respectively in this implementation
        motor=np.array([0,1])

        return x0, edges, fixed_joints, motor, target_idx

    def convert_mech_to_1D(self, x0, edges, fixed_joints, target_idx=None, **kwargs):
        # This function assumes motor to be [0, 1] our random mechanism generator automatically does this
        N = self.N

        # Initialize dictionary to store 1D representation of mechanism
        x = {}

        # Store target node value
        if target_idx is None:
            target_idx = x0.shape[0]-1 # Assume last node is the target if not specified
            
        x["target"] = target_idx

        # Store connectivity matrix in its flattened form
        C = np.zeros((N,N), dtype=bool)
        C[edges[:,0], edges[:,1]] = 1
        C[edges[:,1], edges[:,0]] = 1
       
        for i in range(N):
            for j in range(i):
                x["C" + str(j) + "_" + str(i)] = C[i,j]

        del x["C0_1"]
        
        # Store position matrix in its flattened form
        if x0.shape[0] != N:
            x0 = np.pad(x0, ((0, N - x0.shape[0]), (0, 0)), 'constant', constant_values=0)
            
        for i in range(2*N):
            x["X0" + str(i)] = x0.flatten()[i]

        # Store fixed nodes in boolean vector form
        for i in range(N):
            x["fixed_nodes" + str(i)] = (i in fixed_joints) or (i>=N)

        return x

    def _evaluate(self, x, out, *args, **kwargs):
        #Convert to mechanism representation
        x0, edges, fixed_joints, motor, target_idx = self.convert_1D_to_mech(x)
        
        # Simulate
        distance, material = PROBLEM_TOOLS(x0,
                                edges,
                                fixed_joints,
                                motor,
                                self.target_curve,
                                target_idx=target_idx
                            )

        out["F"] = np.array([distance, material])
        out["G"] = out["F"] - np.array([0.75, 10.0])  # Constraints: distance <= 0.75, material <= 10.0


## 2) Generate Initial Mechanisms Population
### 2.a Randomize Mechanisms (Advanced — Cells 10–11)

In [ ]:

from LINKS.Optimization import MechanismRandomizer
from LINKS.Visualization import MechanismVisualizer


In [ ]:

randomizer = MechanismRandomizer(
    min_size = 6, # smalllest mechanism to sample
    max_size = 14, # largest mechanism to sample
    device='cpu')

visualizer = MechanismVisualizer()


### 2.b Initial Pool Evaluation (Advanced — Cells 14–15)

In [ ]:

# We will prepare per-target populations; here we demonstrate for a single curve index.
# This cell will be reused inside the GA loop below.
curve_index_demo = 1  # as in the advanced example
mechanisms = [randomizer(n=7) for _ in trange(100)]

problem = mechanism_synthesis_optimization(target_curves[curve_index_demo], N=7)

initial_population = [problem.convert_mech_to_1D(**mech) for mech in mechanisms]

class sample_from_random(Sampling):
        def _do(self, problem, n_samples, **kwargs):
                return np.array([initial_population[i%len(initial_population)] for i in range(n_samples)])

F = problem.evaluate(np.array(initial_population))[0]
print(f'Best Distance Performance In random population: {F[:,0].min()}')
print(f'Best Material Performance In random population: {F[:,1].min()}')


## 3) GA Refinement (Advanced — Cell 16)

In [ ]:

algorithm = NSGA2(pop_size=100,
                  sampling=sample_from_random(),
                  mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
                  mutation=PolynomialMutation(prob=0.5),
                  eliminate_duplicates=MixedVariableDuplicateElimination())

results = minimize(problem,
                algorithm,
                ('n_gen', 100),
                verbose=True,
                save_history=True,
                seed=123
                )


### 3.a Save GA Results Per-Curve (consistent with Starter loading pattern)

In [ ]:

# Convert GA results to mechanism dicts and save a numpy file for the demo curve index
# Uses Advanced-provided convert_1D_to_mech (inverse is in class, here we reconstruct from decision vector via that method).
ga_mechanisms = []
if hasattr(results, 'pop') and results.pop is not None:
    # Extract complete population
    for individual in results.pop.get("X"):
        # individual is a dict of mixed variables compatible with convert_1D_to_mech
        x0, edges, fixed_joints, motor, target_idx = problem.convert_1D_to_mech(individual)
        ga_mechanisms.append({
            'x0': x0,
            'edges': edges,
            'fixed_joints': fixed_joints,
            'motor': motor,
            'target_idx': target_idx
        })

np.save(f'ga_results_curve_{curve_index_demo}.npy', ga_mechanisms, allow_pickle=True)
print(f"Saved GA mechanisms for curve {curve_index_demo} -> ga_results_curve_{curve_index_demo}.npy")


## 4) Gradient Tools Demonstration (Starter — Cells 31–35)

In [ ]:

# Compile differentiable tools (Starter pattern)
gradient_tools = DifferentiableTools(
    device='cpu' # device to run the optimization on
)
gradient_tools.compile() # compile the functions for faster runs


In [ ]:

# If we have at least one GA mechanism, demonstrate gradient evaluation on the first one for the demo target
if len(ga_mechanisms) > 0:
    mech0 = ga_mechanisms[0]
    x0 = mech0['x0']
    edges = mech0['edges']
    fixed_joints = mech0['fixed_joints']
    motor = mech0['motor']
    target_curve = target_curves[curve_index_demo]

    distance, material, distance_grad, material_grad = gradient_tools(x0,
                                            edges,
                                            fixed_joints,
                                            motor,
                                            target_curve,
                                            target_idx=mech0.get('target_idx', None)
                                        )

    print(f"Distance to target curve {curve_index_demo}: {distance:.4f}")
    print(f"Material used: {material:.4f}")
    print(f"Distance Gradient shape(s): {[g.shape for g in distance_grad]}")
    print(f"Material Gradient shape(s): {[g.shape for g in material_grad]}")


## 5) Evaluation & Visualization Utilities (Starter — Sections)

In [ ]:

from LINKS.Kinematics import MechanismSolver
from LINKS.Geometry import CurveEngine


In [ ]:

# Example: solve and visualize the first GA mechanism for the demo curve
if len(ga_mechanisms) > 0:
    mech0 = ga_mechanisms[0]
    x0 = mech0['x0']
    edges = mech0['edges']
    fixed_joints = mech0['fixed_joints']
    motor = mech0['motor']

    solver = MechanismSolver(device='cpu')
    solution = solver(x0, edges, fixed_joints, motor)
    print("The shape of the solution is:", solution.shape)  # (num_joints, timesteps, 2)

    # Plot the trajectory of the (last) joint as in Starter
    target_joint_idx = solution.shape[0]-1
    traced_curve = solution[target_joint_idx]

    plt.figure(figsize=(5,5))
    plt.plot(traced_curve[:,0], traced_curve[:,1])
    plt.axis('equal')
    plt.title(f"Joint {target_joint_idx}'s Traced Curve")
    plt.axis('off')
    plt.show()

    # Compare to the demo target curve using CurveEngine (Starter pattern)
    curve_processor = CurveEngine(
        normalize_scale=False,
        device='cpu'
    )
    curve_processor.visualize_alignment(traced_curve, target_curves[curve_index_demo])
    curve_processor.visualize_comparison(traced_curve, target_curves[curve_index_demo])


## 6) Full Run Over All 6 Target Curves (Advanced — Cells 14–16 pattern repeated)

In [ ]:

all_curve_results = {}

for curve_index in range(6):
    print(f"=== Running GA for Target Curve {curve_index} ===")
    mechanisms = [randomizer(n=7) for _ in trange(100)]
    problem = mechanism_synthesis_optimization(target_curves[curve_index], N=7)
    initial_population = [problem.convert_mech_to_1D(**mech) for mech in mechanisms]

    class sample_from_random(Sampling):
            def _do(self, problem, n_samples, **kwargs):
                    return np.array([initial_population[i%len(initial_population)] for i in range(n_samples)])

    # Optional quick evaluation print, as in Advanced
    F = problem.evaluate(np.array(initial_population))[0]
    print(f'Initial pool — best distance: {F[:,0].min():.6f} | best material: {F[:,1].min():.6f}')

    algorithm = NSGA2(pop_size=100,
                      sampling=sample_from_random(),
                      mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
                      mutation=PolynomialMutation(prob=0.5),
                      eliminate_duplicates=MixedVariableDuplicateElimination())

    results = minimize(problem,
                    algorithm,
                    ('n_gen', 100),
                    verbose=True,
                    save_history=True,
                    seed=123
                    )

    # Convert population to mechanism dicts and save
    curve_mechanisms = []
    if hasattr(results, 'pop') and results.pop is not None:
        for individual in results.pop.get("X"):
            x0, edges, fixed_joints, motor, target_idx = problem.convert_1D_to_mech(individual)
            curve_mechanisms.append({
                'x0': x0,
                'edges': edges,
                'fixed_joints': fixed_joints,
                'motor': motor,
                'target_idx': target_idx
            })

    all_curve_results[curve_index] = curve_mechanisms
    np.save(f'ga_results_curve_{curve_index}.npy', curve_mechanisms, allow_pickle=True)
    print(f"Saved GA mechanisms for curve {curve_index} -> ga_results_curve_{curve_index}.npy")


## 7) (Optional) Starter Mechanism Baseline (Starter — Cells 29–35)

In [ ]:

from LINKS.Visualization import MechanismVisualizer  # already imported above; kept for parity

starter_mech = np.load('starter_mechanism.npy',allow_pickle=True).item() #Load mechanism

x0 = starter_mech['x0']
edges = starter_mech['edges']
fixed_joints = starter_mech['fixed_joints']
motor = starter_mech['motor']

plt.figure(figsize=(8,8))
visualizer = MechanismVisualizer()
visualizer(x0, edges, fixed_joints, motor, ax=plt.gca())


In [ ]:

optimization_tools = Tools(
    device='cpu' # device to run the optimization on
)
optimization_tools.compile() # compile the functions for faster runs

gradient_tools = DifferentiableTools(
    device='cpu' # device to run the optimization on
)
gradient_tools.compile() # compile the functions for faster runs


In [ ]:

# Evaluate baseline on one target curve (index 0 as in Starter)
distance, material = optimization_tools(x0,
                                        edges,
                                        fixed_joints,
                                        motor,
                                        target_curves[0],
                                        target_idx=None
                                    ) 

print(f"Distance to target curve 0: {distance:.4f}")
print(f"Material used: {material:.4f}")


In [ ]:

distance, material, distance_grad, material_grad = gradient_tools(x0,
                                        edges,
                                        fixed_joints,
                                        motor,
                                        target_curves[0],
                                        target_idx=None
                                    )

print(f"Distance to target curve 0: {distance:.4f}")
print(f"Material used: {material:.4f}")
print(f"Distance Gradient (sample length): {len(distance_grad)}")
print(f"Material Gradient (sample length): {len(material_grad)}")


## 8) Save Aggregated Results

In [ ]:

# Save all-curve GA results bundle as a single file too
np.save('ga_results_all_curves.npy', all_curve_results, allow_pickle=True)
print("Saved: ga_results_all_curves.npy")
